In [8]:
from langchain.output_parsers import PydanticOutputParser
import os

from dotenv import load_dotenv
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel

load_dotenv()

template = """I have a Google search request: "{question}". Please decompose the question into several subsequent questions where I can pass the answer from the previous question to the next. Each question should only depend on the answer of the previous question. Make sure each subsequent query has a placeholder in square brackets explains what information need to be passed. Make wording of these questions as a Google search request query. Please respond with a maximum of 2 decomposed questions.
Tip: Make sure that questions do not repeat each other.
Tip: Answer in json format.
Tip: Returned json object should contain "questions" field with array of string questions inside.
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model_name="gpt-4-1106-preview")
model.bind(response_format={"type": "json_object"})


class OutputQuestionsSchema(BaseModel):
    questions: list[str]


chain = (
        {"question": RunnablePassthrough()}
        | prompt
        | model
        | PydanticOutputParser(pydantic_object=OutputQuestionsSchema)
)

In [9]:
output_questions_model: OutputQuestionsSchema = chain.invoke("what is the program of political party that win last elections in the Netherlands")

In [10]:
print(output_questions_model)

questions=['What political party won the last elections in the Netherlands?', 'What is the program of the [winning political party] in the Netherlands?']


In [11]:
from lib.rag_internet_search import answer_simple_question
from lib.runnables import google_queries_merger_runnable

answer: str = None
answers: list[str] = []

for question in output_questions_model.questions:
    combined_question = question
    if answer:
        merger_runnable = google_queries_merger_runnable()
        combined_question = merger_runnable.invoke(". ".join([answer, question]))
    
    print(combined_question)
    answer = answer_simple_question(combined_question)
    answers.append(answer)
    print(answer)
        
print(f"Final answer: {answer}")


What political party won the last elections in the Netherlands?
Query: https://en.wikipedia.org/wiki/2021_Dutch_general_election
Query: https://en.wikipedia.org/wiki/2023_Dutch_general_election
Query: https://apnews.com/article/netherlands-election-candidates-prime-minister-f31f57a856f006ff0f2fc4984acaca6b
Query: https://www.nytimes.com/2023/11/23/world/europe/dutch-election-results-far-right-geert-wilders.html
Query: https://www.bbc.com/news/world-europe-67504272
Query: https://www.politico.eu/europe-poll-of-polls/netherlands/
Query: https://www.theguardian.com/us-news/2023/nov/23/first-thing-dutch-election-results-put-geert-wilders-far-right-party-in-lead
Query: https://www.aa.com.tr/en/europe/final-official-results-of-dutch-elections-confirm-wilders-win/3070502
Query: https://www.cnbc.com/2023/11/23/dutch-election-what-comes-next-after-shock-far-right-victory.html
Query: https://time.com/6339173/dutch-election-results-geert-wilders-far-right-netherlands-europe/
The political party t

In [12]:
from langchain_core.output_parsers import StrOutputParser

def generate_qna_template(questions, answers):
    qnas = []
    for q, a in zip(questions, answers):
        qnas.append(f"""
QUESTION:
{q}
ANSWER:
{a}
""")
    return "".join(qnas)


qnas = generate_qna_template(output_questions_model.questions, answers)

print(qnas)


QUESTION:
What political party won the last elections in the Netherlands?
ANSWER:
The political party that won the last elections in the Netherlands is the Party for Freedom (PVV) led by Geert Wilders.

QUESTION:
What is the program of the [winning political party] in the Netherlands?
ANSWER:
The Party for Freedom (Partij voor de Vrijheid; PVV) in the Netherlands is known primarily for promoting anti-immigration and anti-Islamic policies. The PVV, led by Geert Wilders, is an anti-Islam populist party that draws on support from the now defunct List Pim Fortuyn (LPF), which was named after its founder Wilhelmus Fortuyn, who was also an anti-immigration populist and was assassinated in 2002. The PVV has also been quick to capitalize on populist sentiment, particularly in the wake of the euro-zone debt crisis. The party is characterized as Euroskeptic and has been involved in national debates on immigrant amnesty, assimilation, and the clash of Christian and Islamic cultures in the Nether

In [13]:
from operator import itemgetter

template = """
I have a complex question that was split on multiple simple questions and answers. I need to summarise these questions and answers to provide correct answer to the complex question. Summarize the answer only based on provided questions and answers:
{qnas}

Please provide an answer for the complex question:
{question}

Tip: Make sure you provided correct output.
Tip: Make sure you provide just text and do not include it in quotes
Tip: Do not propose google some part of the answer, keep the summary with the answers in questions-answers.
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), model_name="gpt-4-1106-preview")

runnable = (
        {"qnas": itemgetter("qnas"), "question": itemgetter("question")}
        | prompt
        | model
        | StrOutputParser()
)

In [14]:
runnable.invoke({"qnas": qnas, "question": "what is the program of political party that win last elections in the Netherlands"})

'The Party for Freedom (PVV), led by Geert Wilders, won the last elections in the Netherlands. The PVV is primarily known for its anti-immigration and anti-Islamic policies. It draws on support from the legacy of the List Pim Fortuyn (LPF) and has capitalized on populist sentiment, especially during the euro-zone debt crisis. The party is characterized as Euroskeptic and has been involved in national debates concerning immigrant amnesty, assimilation, and the perceived cultural conflicts between Christian and Islamic values in the Netherlands.'